In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import models
import torch.nn.functional as F
import matplotlib.pyplot as plt
import glob
import os
from skimage.transform import resize
import cv2
import itertools
import warnings
warnings.filterwarnings('ignore')
import skimage as sk
from natsort import natsorted

device = torch.device("cuda")
#torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [ ]:
def create_pair_lists(train_test_path):
    
    class_names = [names for names in glob.glob(train_test_path+"/*")]
    #print(class_names)
    
    a = []
    for names in class_names:
        folder = []
        for folders in glob.glob(names+"/*"):
            folder.append(folders)
        a.append(folder)
        

    paired_folders = itertools.zip_longest(*a)
    
    return list(paired_folders)



In [ ]:
def create_sets(folder,label):
    X_ = []
    Y_ = []
    frame_list = []
    num_of_images = 0
    for frame in glob.glob(folder+"/*.jpg"):
        im_array = im_toarray(frame)
        X_.append(im_array)
        Y_.append(label)
        num_of_images += 1
    
    return normalize(np.array(X_)),np.array(Y_),num_of_images

In [ ]:
def im_toarray(im):
    array = plt.imread(im)
    array = resize(array,(224,224,3),anti_aliasing=True)
    return array

In [ ]:
def one_hot(array, num_classes):
    return np.squeeze(np.eye(num_classes)[array.reshape(-1)])

In [ ]:
def normalize(X):
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    X_normalized = (X - mean)/std
    return X_normalized
    

In [ ]:
def array_to_tensor(array):
    tensor = torch.from_numpy(array)
    return tensor

In [ ]:

def model():

    model = models.resnet34(pretrained=True)
    
    for param in model.parameters():
        param.requires_grad = False

    p = 0
    for child in model.children():
        if p > 4:
            for param in child.parameters():
                param.requires_grad = True
        p += 1
    
    return model


In [ ]:
class LSTMCell(nn.Module):
    def __init__(self, flatten_dim, hidden_size, bias):
        super(LSTMCell, self).__init__()
    
        self.flatten_dim = flatten_dim
        self.hidden_size = hidden_size

        self.bias        = bias
        
        self.i2h = nn.Linear(flatten_dim, 4*hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 4*hidden_size, bias=bias)
        
 
        
    def forward(self, x, cur_state):
        
        if cur_state is None:
            cur_state = (Variable(torch.zeros(1, self.hidden_size)).to(device).float(),
                Variable(torch.zeros(1, self.hidden_size)).to(device).float())
        
        x = x.view(1,-1)
        c_cur, h_cur = cur_state
        preact = self.i2h(x) + self.h2h(h_cur)
        #print(preact.size())
        ingate, forgetgate, cellgate, outgate = preact.chunk(4, 1)
        
        
        ingate = F.sigmoid(ingate)
        forgetgate = F.sigmoid(forgetgate)
        cellgate = F.tanh(cellgate)
        outgate = F.sigmoid(outgate)
        
        c_next = (forgetgate * c_cur) + (ingate * cellgate)
        h_next = outgate * F.tanh(c_next)
        
        next_state = (c_next,h_next)
        
        #softmax_out = F.softmax(self.linear(h_next.view(1,-1)),dim=1)
        
        return next_state


In [ ]:
class RESLSTM(nn.Module):
    def __init__(self, original_model):
        super(RESLSTM, self).__init__()
        
        self.cnn = nn.Sequential(*list(original_model.children())[:-1]).to(device).float()
        #self.dropout = nn.Dropout(p=0.5)
        self.lstm1 = LSTMCell(512,512,True).to(device).float()
        self.lstm2 = LSTMCell(512,99,True).to(device).float()
        #self.lstm3 = LSTMCell(128,99,True).to(device).float()

        
    def forward(self,x,state1,state2):
        conv = self.cnn(x)
        c1,h1 = self.lstm1(conv,state1)
        c2,h2 = self.lstm2(h1,state2)
        #c3,h3 = self.lstm3(h2,state3)
        next_state1 = (c1,h1)
        next_state2 = (c2,h2)
        #next_state3 = (c3,h3)
        softmax = F.softmax(h2,dim=1)    
        return next_state1,next_state2,softmax
    

In [ ]:

def calculate_loss(out,real):

    log_loss = torch.mean(-real*torch.log(out))
    mean_loss = torch.mean((out-real)**2)
    
    loss = (mean_loss + log_loss)/2.0

    return log_loss

In [ ]:
def prepare_X_Y(folder_class,target):
    nb_classes = 99
    X,Y,num_of_images = create_sets(folder_class,target)
    X = X.reshape(-1,3,224,224)
    Y = one_hot(Y,nb_classes)
    X,Y = array_to_tensor(X).to(device).float(),array_to_tensor(Y).to(device).float()
    return X,Y,num_of_images

In [ ]:
def train_loader(pair_training,ind):
    train_pairs = list(pair_training[ind])
    notnone = len(train_pairs) - train_pairs.count(None)
    print(str(notnone)+"/"+str(len(train_pairs)))
    target = 0
    
    for folder_class in train_pairs:
        
        if folder_class is not None:
            
            X,Y,num_of_images = prepare_X_Y(folder_class,target)
            yield (folder_class,X,Y,num_of_images,notnone)
            
        target += 1
    

In [ ]:
def main():
    resnet34 = model().to(device).float()
    res_lstm = RESLSTM(resnet34).to(device).float()
    
    pair_training = create_pair_lists(train_test_path= "Frames/Train")
    pair_test =  create_pair_lists(train_test_path= "Frames/Test")
    
    nb_epoch = 25
    lr = 3.e-1
    optim = filter(lambda p: p.requires_grad,res_lstm.parameters())  
    optimizer = torch.optim.RMSprop(optim,lr)
    
    
    for epoch in range(1,nb_epoch+1):
        epoch_loss = 0
        epoch_correct = 0
        epoch_notnone_videos = 0
        epoch_images = 0
        for ind in range(len(pair_training)):
            train_batch = train_loader(pair_training,ind)
            pair_loss = 0
            pair_correct = 0
            total_pair_images = 0
            for folder_class,X,Y,num_of_images,notnone in train_batch:
                #print(folder_class)
                folder_class_loss,folder_correct = train(X,Y,res_lstm,optimizer)
                pair_loss += folder_class_loss
                pair_correct += folder_correct
                total_pair_images += num_of_images
            
            print("pair: "+str(ind))
            print("there are "+str(total_pair_images)+" images in the pair")
            print("pair loss: "+str(pair_loss.item()/total_pair_images))
            print("pair accuracy: "+str(pair_correct)+"/"+str(notnone))
            print("********")
            epoch_loss += pair_loss
            epoch_correct += pair_correct
            epoch_images += total_pair_images
            epoch_notnone_videos += notnone
        
        print("epoch: "+str(epoch))
        print("there are "+str(epoch_images)+" images in the data")
        print("epoch loss: "+str(epoch_loss.item()/epoch_images))
        print("epoch accuracy: "+str(epoch_correct)+"/"+str(epoch_notnone_videos))
        print("---------------------------")

In [ ]:
def train(X,Y,model,optimizer):
    folder_class_loss = 0
    
    state1 = None
    state2 = None
    softmax_average = 0
    
    for f in range(0,X.size()[0]):
        inp = X[f:f+1]
        real = Y[f:f+1]
                        
        state1,state2,softmax_out = model(inp,state1,state2)
        loss = calculate_loss(softmax_out,real)
        folder_class_loss += loss
        softmax_average += softmax_out
        
    folder_class_average_loss = folder_class_loss/X.size(0)
    softmax_average = softmax_average/X.size(0)
    
    correct = 0
    if torch.argmax(softmax_average,dim=1).item() == torch.argmax(real,dim=1).item():
        correct = 1
    
    optimizer.zero_grad()
                    
    folder_class_average_loss.backward()
                    
    optimizer.step()
    
    return folder_class_loss,correct
    
    

In [ ]:
main()

In [ ]:
"""folder1 = "Frames/Train/ApplyLipstick/1"
X1,Y1 = create_sets(folder1,78)
X1 = X1.reshape(-1,3,224,224)
Y1 = one_hot(Y1,99)
X1,Y1 = array_to_tensor(X1).to(device).float(),array_to_tensor(Y1).to(device).float()

folder2 = "Frames/Train/Bowling/1"
X2,Y2 = create_sets(folder2,55)
X2 = X2.reshape(-1,3,224,224)
Y2 = one_hot(Y2,99)
X2,Y2 = array_to_tensor(X2).to(device).float(),array_to_tensor(Y2).to(device).float()

xlist = [X1,X2]
ylist = [Y1,Y2]

optim_cnn = filter(lambda p: p.requires_grad,res_lstm.cnn.parameters())    
optim_lstm1 = filter(lambda p: p.requires_grad,res_lstm.lstm1.parameters()) 
optim_lstm2 = filter(lambda p: p.requires_grad,res_lstm.lstm2.parameters()) 
#optim_lstm3 = filter(lambda p: p.requires_grad,res_lstm.lstm3.parameters()) 

#a = torch.rand(6,3,224,224).to(device).float()
#b = array_to_tensor(one_hot(np.array([3,3,3,3,3,3]),99)).to(device).float()

optimizer_cnn = torch.optim.Adam(optim_cnn,lr=5.e-2)
optimizer_lstm1 = torch.optim.SGD(optim_lstm1,lr=3.e-1)
optimizer_lstm2 = torch.optim.SGD(optim_lstm2,lr=3.e-1)
#optimizer_lstm3 = torch.optim.Adam(optim_lstm3,lr=3.e-2)
X1.is_cuda"""


In [ ]:
"""model_parameters = filter(lambda p: p.requires_grad, res_lstm.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])  
print(params)

loss_list = []
epoch_list = []
for epoch in range(1,501):

    epoch_loss = 0
    
    for l in range(2):
        X = xlist[l]
        Y = ylist[l]
        folder_loss = 0
        
        state1 = None
        state2 = None
        for f in range(0,X.size()[0]):
            inp = X[f:f+1]
            real = Y[f:f+1]
            state1,state2,softmax_out = res_lstm(inp,state1,state2)
            loss,correct = evaluate(softmax_out,real)
            folder_loss += loss
        
        folder_loss = folder_loss/X.size()[0]
    

        optimizer_lstm2.zero_grad()
        optimizer_lstm1.zero_grad()
        optimizer_cnn.zero_grad()

        folder_loss.backward()


        optimizer_lstm2.step()
        optimizer_lstm1.step()
        optimizer_cnn.step()
        
        index = torch.argmax(softmax_out).item()

        
    epoch_loss += folder_loss

    if epoch % 10 == 0:
        print("epoch: "+str(epoch))
        print("loss: "+str(epoch_loss))
        index = torch.argmax(softmax_out).item()
        print(index)
        print(softmax_out[:,index].item()*100)
        print("*************")"""

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""

    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(k=maxk, largest=True, sorted=True)
    pred = pred.t()
    pred[2] = 78
    print(pred)
    print(target)
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    print(correct)
    res = []
    for k in topk:
        print(k)
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [ ]:
y = Y1[0:1,:]
x = X1[0:1,:,:,:]
s,s2,out = res_lstm(x,None,None)

In [ ]:
out_ = torch.argmax(out,dim=1)
real = torch.argmax(y,dim=1)
print(out_,real)
res = accuracy(out,real,(1,5))

In [ ]:
res

In [ ]:
a = torch.rand(3,4,5)

In [ ]:
a.size(0)

In [ ]:
b = [1,2,None,3,None]
b.count(None)